In [2]:
import pandas as pd
import os

In [14]:
def getXY(ids,labels,concat_cols):
    data_icu = True
    concat = False
    X_df=pd.DataFrame()   
    y_df=pd.DataFrame()   
    features=[]
    #print(ids)
    # For every one of the samples in the ./data/csv folder, get the data. 
    for sample in ids:
        # Get different labels depending on whether or not we are looking at ICU data. 
        if data_icu:
            y=labels[labels['stay_id']==sample]['label']
        else:
            y=labels[labels['hadm_id']==sample]['label']

        # Read in the dynamic data. 
        dyn=pd.read_csv('./data/csv/'+str(sample)+'/dynamic.csv',header=[0,1])

        # I do not know what this concat stuff is doing, but I believe it is not what we ae working on. 
        if concat:
            dyn.columns=dyn.columns.droplevel(0)
            dyn=dyn.to_numpy()
            dyn=dyn.reshape(1,-1)
            #print(dyn.shape)
            #print(len(concat_cols))
            dyn_df=pd.DataFrame(data=dyn,columns=concat_cols)
            features=concat_cols

        # This is the only one we are really going to use, for now. 
        else:
            dyn_df=pd.DataFrame()
            #print(dyn)
            for key in dyn.columns.levels[0]:
                #print(sample)                    
                dyn_temp=dyn[key]
                # This is the one that we are using. 
                if data_icu:
                    if ((key=="CHART") or (key=="MEDS")):
                        agg=dyn_temp.aggregate("mean")
                        agg=agg.reset_index()
                    else:
                        agg=dyn_temp.aggregate("max")
                        agg=agg.reset_index()
                else:
                    if ((key=="LAB") or (key=="MEDS")):
                        agg=dyn_temp.aggregate("mean")
                        agg=agg.reset_index()
                    else:
                        agg=dyn_temp.aggregate("max")
                        agg=agg.reset_index()
                if dyn_df.empty:
                    dyn_df=agg
                else:
                    dyn_df=pd.concat([dyn_df,agg],axis=0)
            dyn_df=dyn_df.T
            dyn_df.columns = dyn_df.iloc[0]
            dyn_df=dyn_df.iloc[1:,:]
        # get the static and demo data. 
        stat=pd.read_csv('../MIMIC-IV-Data-Pipeline/data/csv/'+str(sample)+'/static.csv',header=0)
        demo=pd.read_csv('../MIMIC-IV-Data-Pipeline/data/csv/'+str(sample)+'/demo.csv',header=0)

        # Fill up the X_df with the demo data. 
        if X_df.empty:
            X_df=pd.concat([dyn_df,stat],axis=1)
            X_df=pd.concat([X_df,demo],axis=1)
        else:
            X_df=pd.concat([X_df,pd.concat([pd.concat([dyn_df,stat],axis=1),demo],axis=1)],axis=0)

        # Just make y equal to the label (with the labels describing how many 
        if y_df.empty:
            y_df=y
        else:
            y_df=pd.concat([y_df,y],axis=0)
            
    # remove all of the duplicate columns. 
    X_df = X_df.loc[:,~X_df.columns.duplicated()].copy()
    print("X_df",X_df.shape)
    print("y_df",y_df.shape)
    return X_df ,y_df

In [15]:
all_csv = os.listdir("data/csv")
concat_cols = None
labels=pd.read_csv('./data/labels.csv', header=0)

In [16]:
ids = []
for csv in all_csv:
    try:
        ids.append(int(csv))
    except:
        continue

In [17]:
X_df, Y_df = getXY(ids,labels,concat_cols)

X_df (2538, 25)
y_df (2538,)


In [22]:
X_df.to_csv("All_train_X.csv", index=False)
Y_df.to_csv("All_train_Y.csv", index=False)